In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["MLFLOW_TRACKING_URI"] = os.getenv("MLFLOW_TRACKING_URI")
os.environ["MLFLOW_TRACKING_USERNAME"] = os.getenv("MLFLOW_TRACKING_USERNAME")
os.environ["MLFLOW_TRACKING_PASSWORD"] = os.getenv("MLFLOW_TRACKING_PASSWORD")

In [3]:
import os
os.chdir("e:/MLOps/DataScienceProject2.0")

In [4]:
from dataclasses import dataclass   
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [5]:
from src.DataScienceProject.constants import *
from src.DataScienceProject.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri=os.getenv("MLFLOW_TRACKING_URI")
        )
    
        return model_evaluation_config

In [7]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [8]:
class ModelEvaluation:
    def __init__ (self, config: ModelEvaluationConfig):
        self.config = config
        
    
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_tracking_uri(self.config.mlflow_uri) 
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme    
        
        with mlflow.start_run():
            
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            scores = {"rmse":rmse, "mae":mae, "r2": r2}
            
            save_json(path = Path(self.config.metric_file_name), data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)
            
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetWineModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [9]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_evaluation_config()
    
    model_evaluation = ModelEvaluation(config=model_eval_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-09-03 21:34:52,430: INFO: common : yaml file: config\config.yml loaded successfully]
[2025-09-03 21:34:52,459: INFO: common : yaml file: params.yml loaded successfully]
[2025-09-03 21:34:52,468: INFO: common : yaml file: schema.yml loaded successfully]
[2025-09-03 21:34:52,478: INFO: common : created directory at : artifacts]
[2025-09-03 21:34:52,481: INFO: common : created directory at : artifacts/model_evaluation]
[2025-09-03 21:34:52,459: INFO: common : yaml file: params.yml loaded successfully]
[2025-09-03 21:34:52,468: INFO: common : yaml file: schema.yml loaded successfully]
[2025-09-03 21:34:52,478: INFO: common : created directory at : artifacts]
[2025-09-03 21:34:52,481: INFO: common : created directory at : artifacts/model_evaluation]


[2025-09-03 21:34:57,197: INFO: common : json file saved at : artifacts\model_evaluation\metric.json]


2025/09/03 21:35:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticnetWineModel'.
Successfully registered model 'ElasticnetWineModel'.
2025/09/03 21:35:28 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetWineModel, version 1
2025/09/03 21:35:28 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetWineModel, version 1
Created version '1' of model 'ElasticnetWineModel'.
Created version '1' of model 'ElasticnetWineModel'.


🏃 View run bustling-koi-119 at: https://dagshub.com/shashwatsingh0511/DataScienceProject2.0.mlflow/#/experiments/0/runs/47dd2e7ab2464782a1850a7bb5cd8135
🧪 View experiment at: https://dagshub.com/shashwatsingh0511/DataScienceProject2.0.mlflow/#/experiments/0
